In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
from importer import Importer
from df_utils import group_weighted_mean_factory, add_new_entries, get_average_price_df, get_qtd, \
    get_exceptional_earnings_since_2018

raw_df = Importer().process_files()
df = raw_df
df.data = raw_df.data.apply(pd.to_datetime)
df.qtd = df.apply(lambda line: line.qtd if line.compra_venda == "C" else -line.qtd, axis=1)

print(raw_df)

weighted_mean = group_weighted_mean_factory(raw_df, "qtd")

df = raw_df.sort_values("data").groupby(by= ["data", "abbreviation", "compra_venda"], axis=0) \
    .aggregate({"qtd": "sum", "preco": "mean",
                "valor_operacao": "sum", "taxas": "max", "total_ajustado": "sum", "preco_ajustado": weighted_mean})
df = df.reset_index()

# ------------- Extra entries -------------
df = get_exceptional_earnings_since_2018(df)

# to_date = datetime(2022, 12, 31)
to_date = datetime(2022, 4, 24)

average_price_df = get_average_price_df(df, to_date)
# display(average_price_df)
qtd_df = get_qtd(df, to_date)

final_df = average_price_df.merge(qtd_df, on="abbreviation")
final_df.columns = ["preco_medio", "qtd"]
final_df = final_df[final_df.qtd != 0]
print(final_df)

1 de 60 documentos processados.
2 de 60 documentos processados.
3 de 60 documentos processados.
4 de 60 documentos processados.
5 de 60 documentos processados.
6 de 60 documentos processados.
7 de 60 documentos processados.
8 de 60 documentos processados.
9 de 60 documentos processados.
10 de 60 documentos processados.
11 de 60 documentos processados.
12 de 60 documentos processados.
13 de 60 documentos processados.
14 de 60 documentos processados.
15 de 60 documentos processados.
16 de 60 documentos processados.
17 de 60 documentos processados.
18 de 60 documentos processados.
19 de 60 documentos processados.
20 de 60 documentos processados.
21 de 60 documentos processados.
22 de 60 documentos processados.
23 de 60 documentos processados.
24 de 60 documentos processados.
25 de 60 documentos processados.
26 de 60 documentos processados.
27 de 60 documentos processados.
28 de 60 documentos processados.
29 de 60 documentos processados.
30 de 60 documentos processados.
31 de 60 documentos

In [21]:
pos = raw_df[(raw_df.abbreviation=='PSSA3') & (raw_df.compra_venda == 'C')].sort_values("data")
neg = raw_df[(raw_df.abbreviation=='PSSA3') & (raw_df.compra_venda == 'V')].sort_values("data")
# calc.total_ajustado.sum()
total = pos.total_ajustado.sum() - neg.total_ajustado.sum()
total

125841.41999999998

In [ ]:
raw_df.data